In [14]:
# Primero importamos las librerias

import pandas as pd 
import numpy as np
import re 
from tqdm import tqdm  # controlar el tiempo en un loop
import os


# Las librerias del modelo lineal

import statsmodels.api as sm  # regresion lineal utiliza todas las columnas de base de datos 
import statsmodels.formula.api as smf  # regresion lineal usa una formula
from sklearn import datasets, linear_model # modelos
from sklearn.metrics import mean_squared_error, r2_score
from linearmodels.iv import IV2SLS 


import warnings
warnings.filterwarnings('ignore') # eliminar warning messages 


# Exportar tabla a latex

from pystout import pystout

In [6]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install linearmodels

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pystout

Note: you may need to restart the kernel to use updated packages.


In [32]:
#Importamos la base de datos del paper de Serra y  Porte

import pandas as pd

df = pd.read_stata(r'C:\Users\mayco\Documents\GitHub\1ECO35_2023_1\1ECO35_2023_1\data\trabajo_final\replicacion\SerraPorterAEJ.dta')
df

,anonid,took_fall,took_year,tookanother,numeconclass,econmajor,Major,Major_STEM,Major_Business,Major_Finance,...,ecojobs_b,ecojobs_c,ecojobs_d,ecojobs_e,ecojobs_f,ecojobs_g,ecojobs_h,major_current_a,major_current_b,survey_econmajor
0,4.0,0.0,0.0,0.0,0.0,0.0,Mathematics - BS,1.0,0.0,0.0,...,Entrepreneur,Journalist/ Correspondent,Working for marketplace on NPR,Business consultant,,,,Mathematics,,0.0
1,5.0,0.0,0.0,0.0,0.0,0.0,Biological Sciences - BS,1.0,0.0,0.0,...,Tax advisor,Managerial Position,,,,,,Biology,,0.0
2,7.0,1.0,1.0,1.0,13.0,1.0,Economics w/ Fina Appl - BS,0.0,0.0,0.0,...,,,,,,,,,,NaN
3,9.0,0.0,0.0,0.0,0.0,0.0,Journalism - BA,0.0,0.0,0.0,...,,,,,,,,Journalism,Communications,0.0
4,10.0,0.0,0.0,1.0,1.0,0.0,Markets & Culture - BA,0.0,0.0,0.0,...,Investment Banker,,,,,,,Markets and Culture,Economics,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,1386.0,0.0,0.0,0.0,0.0,0.0,Markets & Culture - BA,0.0,0.0,0.0,...,,,,,,,,,,NaN
623,1388.0,0.0,0.0,0.0,0.0,0.0,Economics w/ Fina Appl - PMJ,0.0,0.0,0.0,...,,,,,,,,Business,,0.0
624,1389.0,0.0,0.0,0.0,0.0,0.0,Accounting - BBA,0.0,1.0,0.0,...,Finance,Bank,Government,,,,,Economics,Public Relations,1.0
625,1393.0,0.0,1.0,1.0,10.0,1.0,Economics - BA,0.0,0.0,0.0,...,Professor,Economist,,,,,,Entrepreneurship,Finance Economics,0.0


In [33]:
# implemntamos las summmary statistics

df.describe()

# Tipo de variables

df.info()

df.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 627 entries, 0 to 626
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   anonid            627 non-null    float32
 1   took_fall         627 non-null    float32
 2   took_year         627 non-null    float32
 3   tookanother       627 non-null    float32
 4   numeconclass      627 non-null    float32
 5   econmajor         627 non-null    float32
 6   Major             627 non-null    object 
 7   Major_STEM        627 non-null    float32
 8   Major_Business    627 non-null    float32
 9   Major_Finance     627 non-null    float32
 10  Major_Marketing   627 non-null    float32
 11  Major_SocSc       627 non-null    float32
 12  Major_Arts        627 non-null    float32
 13  Major_Comm        627 non-null    float32
 14  Major_Hum         627 non-null    float32
 15  female            627 non-null    float32
 16  greek             501 non-null    float64
 1

anonid              float32
took_fall           float32
took_year           float32
tookanother         float32
numeconclass        float32
econmajor           float32
Major                object
Major_STEM          float32
Major_Business      float32
Major_Finance       float32
Major_Marketing     float32
Major_SocSc         float32
Major_Arts          float32
Major_Comm          float32
Major_Hum           float32
female              float32
greek               float64
econ_hs             float32
varsity             float64
class_fe            float32
class_fe2           float32
yr_2016             float32
treatment_class     float32
treat2016           float32
female_prof         float32
instate             float32
freshman            float32
american            float32
ACumGPA             float64
gradePrinciples     float32
small_class         float32
grade3301           float32
fem2016             float32
femtreat16          float32
female_treat        float32
Testscore3301       

In [39]:
# la primera forma de presentar la tabla de estaditica descriptiva de las variables
table1.describe()

,american,instate,freshman,ACumGPA,greek,econ_hs,varsity
count,627.000000,627.000000,627.000000,627.000000,501.000000,501.000000,500.000000
mean,0.902711,0.221691,0.864434,3.432091,0.642715,0.578842,0.072000
std,0.296587,0.415717,0.342602,0.435175,0.479679,0.494239,0.258747
min,0.000000,0.000000,0.000000,1.200000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,3.229000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,3.500000,1.000000,1.000000,0.000000
75%,1.000000,0.000000,1.000000,3.740000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000


In [44]:
#Ajustamos la tabla para obtener los datos estadisticos  que nos pide el ejercicio
summary_table = table1.describe().loc[["mean","std","count", "min","max"]]
summary_table

,american,instate,freshman,ACumGPA,greek,econ_hs,varsity
mean,0.902711,0.221691,0.864434,3.432091,0.642715,0.578842,0.072000
std,0.296587,0.415717,0.342602,0.435175,0.479679,0.494239,0.258747
count,627.000000,627.000000,627.000000,627.000000,501.000000,501.000000,500.000000
min,0.000000,0.000000,0.000000,1.200000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000


In [45]:
# Ahora, usamos la trasposicion para darle una mejor apariencia
summary_table = table1.describe().loc[["mean","std","count", "min","max"]].T
summary_table

,mean,std,count,min,max
american,0.902711,0.296587,627.0,0.0,1.0
instate,0.221691,0.415717,627.0,0.0,1.0
freshman,0.864434,0.342602,627.0,0.0,1.0
ACumGPA,3.432091,0.435175,627.0,1.2,4.0
greek,0.642715,0.479679,501.0,0.0,1.0
econ_hs,0.578842,0.494239,501.0,0.0,1.0
varsity,0.072000,0.258747,500.0,0.0,1.0


In [55]:
# Revisamos las columnas
table1.columns

Index(['american', 'instate', 'freshman', 'ACumGPA', 'greek', 'econ_hs',
       'varsity'],
      dtype='object')

In [56]:
#Implementamos los nombres completos de las variables
table1.columns

new_names = ["American student","In-state student","Freshman","Cumulative GPA"," Belongs to soroty","Took econ in high school","Athlete"]

dict( zip( table1.columns, new_names) )

{'american': 'American student',
 'instate': 'In-state student',
 'freshman': 'Freshman',
 'ACumGPA': 'Cumulative GPA',
 'greek': ' Belongs to soroty',
 'econ_hs': 'Took econ in high school',
 'varsity': 'Athlete'}

In [57]:
# Ahora, implemntamos los nombres completos a los test estadisticos

index_nuevos_nombres = dict( zip( table1.columns, new_names) )

columns_nuevos_nombres = {
    "mean": "Mean",
    "std": "Standard Deviation",
    "min": "Min",
    "max": "Max",
    "count": "Observations",
}

# Renombramos columnas y filas
summary_table.rename(index=index_nuevos_nombres, columns=columns_nuevos_nombres, inplace=True)

In [58]:
# Obtenemos la nueva tabla mejorada
summary_table

,Mean,Standard Deviation,Observations,Min,Max
American student,0.902711,0.296587,627.0,0.0,1.0
In-state student,0.221691,0.415717,627.0,0.0,1.0
Freshman,0.864434,0.342602,627.0,0.0,1.0
Cumulative GPA,3.432091,0.435175,627.0,1.2,4.0
Belongs to soroty,0.642715,0.479679,501.0,0.0,1.0
Took econ in high school,0.578842,0.494239,501.0,0.0,1.0
Athlete,0.072000,0.258747,500.0,0.0,1.0


In [208]:
# Finalmente, exportamos la tabla a LaTeX

summary_table.style.format(subset="Mean", precision=2).format(subset="Standard Deviation", precision=2)\
.format(subset="Min", precision=2).format(subset="Max", precision=2).format(subset="Observations", precision=0)\
.to_latex(
    r"..\..\Trabajo_final\Grupo9\estadísticas_descriptivas.tex",
caption="Descriptive Statistics",
    column_format = "lccccc"   
) 

In [61]:
# En la segunda parte del ejercicio debemos replicar tablas

# Tabla 3 
## Primera regresion

y = df['took_year']

# agregamos la constante
## "took_year": Took Micro within year
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class"]])
X


,const,treat2016,yr_2016,treatment_class
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0
...,...,...,...,...
622,1.0,1.0,1.0,1.0
623,1.0,1.0,1.0,1.0
624,1.0,0.0,1.0,0.0
625,1.0,1.0,1.0,1.0


In [62]:
# Utilizamos la función Homocedastica 

ols_model = sm.OLS(y, X).fit()

In [63]:
ols_model.summary().tables[0]

Dep. Variable:,took_year,R-squared:,0.008
Model:,OLS,Adj. R-squared:,0.003
Method:,Least Squares,F-statistic:,1.679
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.170
Time:,19:29:00,Log-Likelihood:,-211.88
No. Observations:,627,AIC:,431.8
Df Residuals:,623,BIC:,449.5
Df Model:,3,,
Covariance Type:,nonrobust,,


In [64]:
ols_model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.1474,0.027,5.411,0.000,0.094,0.201
treat2016,0.1150,0.055,2.101,0.036,0.007,0.222
yr_2016,-0.0401,0.037,-1.073,0.284,-0.113,0.033
treatment_class,-0.0377,0.038,-0.990,0.323,-0.112,0.037


In [66]:
#Encontramos el standar error y la significancia 
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:              took_year   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.679
Date:                Sat, 15 Jul 2023   Prob (F-statistic):              0.170
Time:                        19:30:43   Log-Likelihood:                -211.88
No. Observations:                 627   AIC:                             431.8
Df Residuals:                     623   BIC:                             449.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1474      0.027     

In [69]:
# Enocntramos el error estándar robusto de Huber-White

ols_model_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model_rb.summary())

                            OLS Regression Results                            
Dep. Variable:              took_year   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     16.50
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           0.000220
Time:                        19:32:08   Log-Likelihood:                -211.88
No. Observations:                 627   AIC:                             431.8
Df Residuals:                     623   BIC:                             449.5
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1474      0.014     

In [70]:
# Accedemos a la información de la tabla

ols_model_rb.summary2()

ols_model_rb.summary2().tables[1]

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
const,0.147436,0.014305,10.306614,6.577837e-25,0.119399,0.175473
treat2016,0.114951,0.037715,3.047907,2.304413e-03,0.041031,0.188870
yr_2016,-0.040091,0.026968,-1.486637,1.371107e-01,-0.092947,0.012765
treatment_class,-0.037680,0.025463,-1.479803,1.389257e-01,-0.087586,0.012226


In [75]:
# Segunda regresión, la cual contiene  variable de control

y = df['took_year']

# agregamos la constante

X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X

,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [77]:
# Utilizamos la función Homocedastica 
ols_model1 = sm.OLS(y, X).fit()

In [78]:
ols_model1.summary().tables[0]

Dep. Variable:,took_year,R-squared:,0.035
Model:,OLS,Adj. R-squared:,0.020
Method:,Least Squares,F-statistic:,2.250
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.0139
Time:,19:36:26,Log-Likelihood:,-203.16
No. Observations:,627,AIC:,428.3
Df Residuals:,616,BIC:,477.2
Df Model:,10,,
Covariance Type:,nonrobust,,


In [80]:
#Encontramos el standar error y la significancia
ols_model1.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.3720,0.126,2.943,0.003,0.124,0.620
treat2016,0.1122,0.055,2.032,0.043,0.004,0.221
yr_2016,-0.0486,0.037,-1.304,0.193,-0.122,0.025
treatment_class,-0.0473,0.038,-1.237,0.217,-0.123,0.028
female_prof,0.0357,0.028,1.262,0.207,-0.020,0.091
instate,-0.0032,0.033,-0.097,0.923,-0.068,0.062
freshman,0.0149,0.041,0.363,0.717,-0.066,0.095
american,-0.1402,0.047,-2.971,0.003,-0.233,-0.048
ACumGPA,-0.0846,0.046,-1.836,0.067,-0.175,0.006
gradePrinciples,0.0600,0.023,2.622,0.009,0.015,0.105


In [82]:
#Encontramos el standar error y la significancia

print(ols_model1.summary())

                            OLS Regression Results                            
Dep. Variable:              took_year   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     2.250
Date:                Sat, 15 Jul 2023   Prob (F-statistic):             0.0139
Time:                        19:41:29   Log-Likelihood:                -203.16
No. Observations:                 627   AIC:                             428.3
Df Residuals:                     616   BIC:                             477.2
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.3720      0.126     

In [84]:
# Encontramos el satandar error robusto de Huber-White

ols_model1_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model1_rb.summary())

                            OLS Regression Results                            
Dep. Variable:              took_year   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     12.67
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           0.000114
Time:                        19:42:52   Log-Likelihood:                -203.16
No. Observations:                 627   AIC:                             428.3
Df Residuals:                     616   BIC:                             477.2
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.3720      0.099     

In [85]:
## Tercera regresión

y = df['tookanother']

# agregamos la constante
## "tookanother": Took another econ class
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016

X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class"]])
X

,const,treat2016,yr_2016,treatment_class
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0
...,...,...,...,...
622,1.0,1.0,1.0,1.0
623,1.0,1.0,1.0,1.0
624,1.0,0.0,1.0,0.0
625,1.0,1.0,1.0,1.0


In [87]:
# Utilizamos la función Homocedastica 

ols_model2 = sm.OLS(y, X).fit()

In [88]:
ols_model2.summary().tables[0]

Dep. Variable:,tookanother,R-squared:,0.010
Model:,OLS,Adj. R-squared:,0.006
Method:,Least Squares,F-statistic:,2.160
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.0916
Time:,19:48:46,Log-Likelihood:,-330.66
No. Observations:,627,AIC:,669.3
Df Residuals:,623,BIC:,687.1
Df Model:,3,,
Covariance Type:,nonrobust,,


In [89]:
ols_model2.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.2372,0.033,7.202,0.000,0.173,0.302
treat2016,0.1576,0.066,2.384,0.017,0.028,0.287
yr_2016,-0.0620,0.045,-1.373,0.170,-0.151,0.027
treatment_class,-0.0482,0.046,-1.047,0.296,-0.138,0.042


In [91]:
#Encontramos el standar error y la significancia
print(ols_model2.summary())

                            OLS Regression Results                            
Dep. Variable:            tookanother   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.160
Date:                Sat, 15 Jul 2023   Prob (F-statistic):             0.0916
Time:                        19:49:45   Log-Likelihood:                -330.66
No. Observations:                 627   AIC:                             669.3
Df Residuals:                     623   BIC:                             687.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.2372      0.033     

In [93]:
# Encontramos el standar Error robusto de Huber-White

ols_model2_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model2_rb.summary())

                            OLS Regression Results                            
Dep. Variable:            tookanother   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     10.04
Date:                Sat, 15 Jul 2023   Prob (F-statistic):            0.00176
Time:                        19:51:23   Log-Likelihood:                -330.66
No. Observations:                 627   AIC:                             669.3
Df Residuals:                     623   BIC:                             687.1
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.2372      0.018     

In [95]:
## Cuarta regresión, contiene variables de control

y = df['tookanother']

# agregamos la constante
## "tookanother": Took another econ class
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X



,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [96]:
# Función Homocedastica 

ols_model3 = sm.OLS(y, X).fit()

In [97]:
ols_model3.summary().tables[0]

Dep. Variable:,tookanother,R-squared:,0.049
Model:,OLS,Adj. R-squared:,0.034
Method:,Least Squares,F-statistic:,3.172
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.000558
Time:,19:52:37,Log-Likelihood:,-318.16
No. Observations:,627,AIC:,658.3
Df Residuals:,616,BIC:,707.2
Df Model:,10,,
Covariance Type:,nonrobust,,


In [98]:
ols_model3.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.8032,0.152,5.290,0.000,0.505,1.101
treat2016,0.1389,0.066,2.093,0.037,0.009,0.269
yr_2016,-0.0668,0.045,-1.493,0.136,-0.155,0.021
treatment_class,-0.0615,0.046,-1.337,0.182,-0.152,0.029
female_prof,0.0231,0.034,0.678,0.498,-0.044,0.090
instate,-0.0192,0.040,-0.483,0.629,-0.097,0.059
freshman,-0.0154,0.049,-0.312,0.755,-0.112,0.081
american,-0.2090,0.057,-3.686,0.000,-0.320,-0.098
ACumGPA,-0.1370,0.055,-2.476,0.014,-0.246,-0.028
gradePrinciples,0.0420,0.028,1.526,0.128,-0.012,0.096


In [99]:
ols_model3.summary().tables[2]

Omnibus:,110.999,Durbin-Watson:,1.928
Prob(Omnibus):,0.000,Jarque-Bera (JB):,174.915
Skew:,1.294,Prob(JB):,1.04e-38
Kurtosis:,2.986,Cond. No.,49.5


In [100]:
print(ols_model3.summary())

                            OLS Regression Results                            
Dep. Variable:            tookanother   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     3.172
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           0.000558
Time:                        19:53:42   Log-Likelihood:                -318.16
No. Observations:                 627   AIC:                             658.3
Df Residuals:                     616   BIC:                             707.2
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.8032      0.152     

In [102]:
# Encontramos standar error rosbuto  de Huber-White

ols_model3_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model3_rb.summary())

                            OLS Regression Results                            
Dep. Variable:            tookanother   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     6.235
Date:                Sat, 15 Jul 2023   Prob (F-statistic):            0.00281
Time:                        19:54:27   Log-Likelihood:                -318.16
No. Observations:                 627   AIC:                             658.3
Df Residuals:                     616   BIC:                             707.2
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.8032      0.176     

In [103]:
# Variables explicativa a mostrarse en la tabla

explicativas = ['treat2016','yr_2016','treatment_class','const']

# etiquetas para las variables

etiquetas = ['Treatment class × 2016','Year 2016','Treatment class (in 2015)','Constant']


labels = dict(zip(explicativas,etiquetas))
labels 

{'treat2016': 'Treatment class × 2016',
 'yr_2016': 'Year 2016',
 'treatment_class': 'Treatment class (in 2015)',
 'const': 'Constant'}

In [213]:
# Pasamos la tabla 3 a Latex

pystout(models = [ols_model,ols_model1,ols_model2,ols_model3], file=r"..\..\Trabajo_final\Grupo9\tabla3.tex", digits=3,
        endog_names=['Took Micro within year','Took Micro within year','Took another econ class','Took another econ class'],
        exogvars =explicativas ,  
        varlabels = labels,
        modstat={'nobs':'Observations'}, 
        addrows={'Controls':['No','Yes','No','Yes']},  
        addnotes=['Notes: LPM regressions. We report wild bootstrap cluster p-values in parentheses and wild bootstrap cluster 95', 
                  'percent confidence intervals in square brackets, generated using boottest command in Stata 14 (Roodman et al.', 
                  '2019) for standard errors clustered at the class level (12 clusters). In columns 1 and 2, the dependent variable is a', 
                  'dummy equal to one if the student took Intermediate Micro the year after the Principles class. In columns 3 and 4,',
                  'the dependent variable is a dummy equal to one if the student enrolled in another economics class at any time after', 
                  'the Principles class (up to January 2019).', 
                 '* Significantly different from zero at 90 percent confidence.',
                 '** Significantly different from zero at 95 percent confidence.',
                 '*** Significantly different from zero at 99 percent confidence.'],
        title='Treatment Effects on Intermediate Outcomes',
        stars={.1:'*',.05:'**',.01:'***'}
       )

In [106]:
# Tabla 4 
# Primera regresión

y = df['numeconclass']

# agregamos la constante
## "numeconclass": Number of econ classes taken
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class"]])
X


,const,treat2016,yr_2016,treatment_class
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0
...,...,...,...,...
622,1.0,1.0,1.0,1.0
623,1.0,1.0,1.0,1.0
624,1.0,0.0,1.0,0.0
625,1.0,1.0,1.0,1.0


In [107]:
# Utilizamos Función Homocedastica
ols_model4 = sm.OLS(y, X).fit()

In [108]:
ols_model4.summary().tables[0]

Dep. Variable:,numeconclass,R-squared:,0.006
Model:,OLS,Adj. R-squared:,0.001
Method:,Least Squares,F-statistic:,1.290
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.277
Time:,19:57:39,Log-Likelihood:,-1505.4
No. Observations:,627,AIC:,3019.
Df Residuals:,623,BIC:,3037.
Df Model:,3,,
Covariance Type:,nonrobust,,


In [109]:
ols_model4.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,1.0256,0.214,4.783,0.000,0.605,1.447
treat2016,0.6916,0.431,1.606,0.109,-0.154,1.537
yr_2016,-0.1725,0.294,-0.587,0.558,-0.750,0.405
treatment_class,-0.1293,0.300,-0.432,0.666,-0.718,0.459


In [111]:
print(ols_model4.summary())

                            OLS Regression Results                            
Dep. Variable:           numeconclass   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.290
Date:                Sat, 15 Jul 2023   Prob (F-statistic):              0.277
Time:                        19:59:00   Log-Likelihood:                -1505.4
No. Observations:                 627   AIC:                             3019.
Df Residuals:                     623   BIC:                             3037.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.0256      0.214     

In [113]:
# Encontramos standar error  robusto de Huber-White

ols_model4_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model4_rb.summary())

                            OLS Regression Results                            
Dep. Variable:           numeconclass   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.964
Date:                Sat, 15 Jul 2023   Prob (F-statistic):             0.0385
Time:                        19:59:27   Log-Likelihood:                -1505.4
No. Observations:                 627   AIC:                             3019.
Df Residuals:                     623   BIC:                             3037.
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.0256      0.112     

In [115]:
#Segunda regresión, contiene variable de control

y = df['numeconclass']

# agregamos la constante

## "numeconclass": Number of econ classes taken
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X



,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [116]:
# Utilizamos Función Homocedastica 

ols_model5 = sm.OLS(y, X).fit()

In [117]:
ols_model5.summary().tables[0]

Dep. Variable:,numeconclass,R-squared:,0.064
Model:,OLS,Adj. R-squared:,0.049
Method:,Least Squares,F-statistic:,4.210
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,1.14e-05
Time:,20:05:15,Log-Likelihood:,-1486.6
No. Observations:,627,AIC:,2995.
Df Residuals:,616,BIC:,3044.
Df Model:,10,,
Covariance Type:,nonrobust,,


In [118]:
ols_model5.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,5.8037,0.979,5.929,0.000,3.881,7.726
treat2016,0.5220,0.428,1.220,0.223,-0.318,1.362
yr_2016,-0.1890,0.289,-0.655,0.513,-0.756,0.378
treatment_class,-0.2058,0.296,-0.694,0.488,-0.788,0.376
female_prof,0.0300,0.219,0.137,0.891,-0.400,0.460
instate,0.0103,0.255,0.040,0.968,-0.491,0.512
freshman,-0.1340,0.317,-0.422,0.673,-0.757,0.489
american,-1.6778,0.365,-4.591,0.000,-2.395,-0.960
ACumGPA,-1.2725,0.357,-3.566,0.000,-1.973,-0.572
gradePrinciples,0.4540,0.177,2.560,0.011,0.106,0.802


In [119]:
ols_model5.summary().tables[2]

Omnibus:,352.556,Durbin-Watson:,2.126
Prob(Omnibus):,0.000,Jarque-Bera (JB):,1868.524
Skew:,2.635,Prob(JB):,0.00
Kurtosis:,9.614,Cond. No.,49.5


In [120]:
print(ols_model5.summary())


                            OLS Regression Results                            
Dep. Variable:           numeconclass   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     4.210
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           1.14e-05
Time:                        20:06:49   Log-Likelihood:                -1486.6
No. Observations:                 627   AIC:                             2995.
Df Residuals:                     616   BIC:                             3044.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               5.8037      0.979     

In [122]:
# Encontramos standar Error robusto de Huber-White

ols_model5_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model5_rb.summary())

                            OLS Regression Results                            
Dep. Variable:           numeconclass   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     30.57
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           1.37e-06
Time:                        20:07:13   Log-Likelihood:                -1486.6
No. Observations:                 627   AIC:                             2995.
Df Residuals:                     616   BIC:                             3044.
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               5.8037      1.266     

In [125]:
#Tercera regresión

y = df['econmajor']

# agregamos la constante
## "econmajor": Major in economics
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class"]])
X


,const,treat2016,yr_2016,treatment_class
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0
...,...,...,...,...
622,1.0,1.0,1.0,1.0
623,1.0,1.0,1.0,1.0
624,1.0,0.0,1.0,0.0
625,1.0,1.0,1.0,1.0


In [126]:
# Utilizamos Función Homocedastica 

ols_model6 = sm.OLS(y, X).fit()

In [127]:

ols_model6.summary().tables[0]

Dep. Variable:,econmajor,R-squared:,0.009
Model:,OLS,Adj. R-squared:,0.005
Method:,Least Squares,F-statistic:,1.944
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.121
Time:,20:08:19,Log-Likelihood:,-133.19
No. Observations:,627,AIC:,274.4
Df Residuals:,623,BIC:,292.1
Df Model:,3,,
Covariance Type:,nonrobust,,


In [128]:
ols_model6.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.1026,0.024,4.267,0.000,0.055,0.150
treat2016,0.0980,0.048,2.031,0.043,0.003,0.193
yr_2016,-0.0235,0.033,-0.712,0.477,-0.088,0.041
treatment_class,-0.0233,0.034,-0.694,0.488,-0.089,0.043


In [131]:
print(ols_model6.summary())

                            OLS Regression Results                            
Dep. Variable:              econmajor   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.944
Date:                Sat, 15 Jul 2023   Prob (F-statistic):              0.121
Time:                        21:57:55   Log-Likelihood:                -133.19
No. Observations:                 627   AIC:                             274.4
Df Residuals:                     623   BIC:                             292.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1026      0.024     

In [132]:
# Encontramos standar Error robusto de Huber-White

ols_model6_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model6_rb.summary())

                            OLS Regression Results                            
Dep. Variable:              econmajor   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     6.719
Date:                Sat, 15 Jul 2023   Prob (F-statistic):            0.00770
Time:                        21:58:14   Log-Likelihood:                -133.19
No. Observations:                 627   AIC:                             274.4
Df Residuals:                     623   BIC:                             292.1
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1026      0.011     

In [ ]:
# Cuarta regresión, contiene  variable de control)

y = repdata['econmajor']

# agregamos la constante
## "econmajor": Major in economics
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(repdata.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X


In [133]:
# Utilizamos función Homocedastica 

ols_model7 = sm.OLS(y, X).fit()

In [134]:
ols_model7.summary().tables[0]

Dep. Variable:,econmajor,R-squared:,0.009
Model:,OLS,Adj. R-squared:,0.005
Method:,Least Squares,F-statistic:,1.944
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.121
Time:,22:01:29,Log-Likelihood:,-133.19
No. Observations:,627,AIC:,274.4
Df Residuals:,623,BIC:,292.1
Df Model:,3,,
Covariance Type:,nonrobust,,


In [135]:
ols_model7.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.1026,0.024,4.267,0.000,0.055,0.150
treat2016,0.0980,0.048,2.031,0.043,0.003,0.193
yr_2016,-0.0235,0.033,-0.712,0.477,-0.088,0.041
treatment_class,-0.0233,0.034,-0.694,0.488,-0.089,0.043


In [137]:
print(ols_model7.summary())

                            OLS Regression Results                            
Dep. Variable:              econmajor   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.944
Date:                Sat, 15 Jul 2023   Prob (F-statistic):              0.121
Time:                        22:02:14   Log-Likelihood:                -133.19
No. Observations:                 627   AIC:                             274.4
Df Residuals:                     623   BIC:                             292.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1026      0.024     

In [138]:
# Encontramos standar Error  robusto de Huber-White

ols_model7_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model7_rb.summary())

                            OLS Regression Results                            
Dep. Variable:              econmajor   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     6.719
Date:                Sat, 15 Jul 2023   Prob (F-statistic):            0.00770
Time:                        22:02:32   Log-Likelihood:                -133.19
No. Observations:                 627   AIC:                             274.4
Df Residuals:                     623   BIC:                             292.1
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1026      0.011     

In [139]:
# Variabkes explicativa a mostrarse en la tabla

explicativas = ['treat2016','yr_2016','treatment_class','const']

# etiquetas para las variables 

etiquetas = ['Treatment class × 2016','Year 2016','Treatment class (in 2015)','Constant']


labels = dict(zip(explicativas,etiquetas))
labels 

{'treat2016': 'Treatment class × 2016',
 'yr_2016': 'Year 2016',
 'treatment_class': 'Treatment class (in 2015)',
 'const': 'Constant'}

In [212]:
#pasamos la tabla 4 a latex
pystout(models = [ols_model4,ols_model5,ols_model6,ols_model7], file=r"..\..\Trabajo_final\Grupo9\tabla4.tex", digits=3,
        endog_names=['Number of econ classes taken','Number of econ classes taken','Major in economics','Major in economics'],
        exogvars =explicativas ,  
        varlabels = labels,
        modstat={'nobs':'Observations'}, 
        addrows={'Controls':['No','Yes','No','Yes']},  
        addnotes=['Notes: Columns 1–2, OLS regressions; columns 3–4, LPM regressions. We report wild bootstrap cluster p-values\
                  in parentheses and wild bootstrap cluster 95 percent confidence intervals in square brackets, generated using\
                  boottest command in Stata 14 (Roodman et al. 2019) for standard errors clustered at the class level (12 clusters).\
                  In columns 1 and 2, the dependent variable is the number of economics classes taken after the Principles class. In\
                  columns 3 and 4, the dependent variable is a dummy equal to one if the student majored in economics (or declared\
                  the economics major, if the student has not graduated yet).', 
                 '* Significantly different from zero at 90 percent confidence.',
                 '** Significantly different from zero at 95 percent confidence.',
                 '*** Significantly different from zero at 99 percent confidence.'],
        title='Treatment Effects on Final Outcomes',
        stars={.1:'*',.05:'**',.01:'***'}
       )

In [141]:
# Tabla 5
## corremos la primera regresión (incluye variable de control)

y = df['Major_STEM']

# agregamos la constante
## "Major_STEM": Major STEM
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X


,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [209]:
# Utilizamos la Función Homocedastica 

ols_model8 = sm.OLS(y, X).fit()

In [143]:
ols_model8.summary().tables[0]

Dep. Variable:,Major_STEM,R-squared:,0.030
Model:,OLS,Adj. R-squared:,0.014
Method:,Least Squares,F-statistic:,1.909
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.0413
Time:,22:04:22,Log-Likelihood:,-159.54
No. Observations:,627,AIC:,341.1
Df Residuals:,616,BIC:,389.9
Df Model:,10,,
Covariance Type:,nonrobust,,


In [144]:

ols_model8.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.2453,0.118,2.081,0.038,0.014,0.477
treat2016,0.0134,0.052,0.260,0.795,-0.088,0.115
yr_2016,0.0027,0.035,0.078,0.938,-0.066,0.071
treatment_class,-0.0051,0.036,-0.143,0.886,-0.075,0.065
female_prof,-0.0089,0.026,-0.337,0.736,-0.061,0.043
instate,0.0308,0.031,1.000,0.318,-0.030,0.091
freshman,-0.1419,0.038,-3.711,0.000,-0.217,-0.067
american,-0.0678,0.044,-1.539,0.124,-0.154,0.019
ACumGPA,0.0089,0.043,0.206,0.837,-0.076,0.093
gradePrinciples,0.0043,0.021,0.202,0.840,-0.038,0.046


In [145]:
ols_model8.summary().tables[2]

Omnibus:,291.393,Durbin-Watson:,1.943
Prob(Omnibus):,0.000,Jarque-Bera (JB):,964.417
Skew:,2.351,Prob(JB):,3.80e-210
Kurtosis:,6.849,Cond. No.,49.5


In [146]:

print(ols_model8.summary())

                            OLS Regression Results                            
Dep. Variable:             Major_STEM   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.909
Date:                Sat, 15 Jul 2023   Prob (F-statistic):             0.0413
Time:                        22:04:53   Log-Likelihood:                -159.54
No. Observations:                 627   AIC:                             341.1
Df Residuals:                     616   BIC:                             389.9
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.2453      0.118     

In [148]:


ols_model8_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model8_rb.summary())

                            OLS Regression Results                            
Dep. Variable:             Major_STEM   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     6.932
Date:                Sat, 15 Jul 2023   Prob (F-statistic):            0.00179
Time:                        22:05:13   Log-Likelihood:                -159.54
No. Observations:                 627   AIC:                             341.1
Df Residuals:                     616   BIC:                             389.9
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.2453      0.142     

In [149]:

## Segunda regresión, contiene  variable de control

y = df['Major_Finance']

# agregamos la constante
## "Major_Finance": Major finance
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X




,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [150]:

ols_model9 = sm.OLS(y, X).fit()

In [151]:
ols_model9.summary().tables[0]

Dep. Variable:,Major_Finance,R-squared:,0.052
Model:,OLS,Adj. R-squared:,0.036
Method:,Least Squares,F-statistic:,3.345
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.000297
Time:,22:06:26,Log-Likelihood:,-224.50
No. Observations:,627,AIC:,471.0
Df Residuals:,616,BIC:,519.8
Df Model:,10,,
Covariance Type:,nonrobust,,


In [152]:
ols_model9.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.3188,0.131,-2.438,0.015,-0.576,-0.062
treat2016,0.0782,0.057,1.368,0.172,-0.034,0.190
yr_2016,-0.0573,0.039,-1.485,0.138,-0.133,0.018
treatment_class,-0.0371,0.040,-0.937,0.349,-0.115,0.041
female_prof,0.0268,0.029,0.917,0.360,-0.031,0.084
instate,-0.0110,0.034,-0.323,0.747,-0.078,0.056
freshman,0.1323,0.042,3.119,0.002,0.049,0.216
american,0.0241,0.049,0.494,0.621,-0.072,0.120
ACumGPA,0.0726,0.048,1.522,0.128,-0.021,0.166
gradePrinciples,0.0346,0.024,1.462,0.144,-0.012,0.081


In [154]:
print(ols_model9.summary())

                            OLS Regression Results                            
Dep. Variable:          Major_Finance   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     3.345
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           0.000297
Time:                        22:07:49   Log-Likelihood:                -224.50
No. Observations:                 627   AIC:                             471.0
Df Residuals:                     616   BIC:                             519.8
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.3188      0.131     

In [155]:
# Encontramos standar Error  robusto de Huber-White

ols_model9_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model9_rb.summary())

                            OLS Regression Results                            
Dep. Variable:          Major_Finance   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     61.41
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           3.46e-08
Time:                        22:08:06   Log-Likelihood:                -224.50
No. Observations:                 627   AIC:                             471.0
Df Residuals:                     616   BIC:                             519.8
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.3188      0.149     

In [156]:
#Tercera regresión,contiene variable de control)

y = df['Major_Business']

# agregamos la constante
## "Major_Business": Major business
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X


,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [157]:
# Utilizamos Función Homocedastica 
ols_model10 = sm.OLS(y, X).fit()

In [158]:
ols_model10.summary().tables[0]

Dep. Variable:,Major_Business,R-squared:,0.022
Model:,OLS,Adj. R-squared:,0.007
Method:,Least Squares,F-statistic:,1.416
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.169
Time:,22:08:48,Log-Likelihood:,-319.96
No. Observations:,627,AIC:,661.9
Df Residuals:,616,BIC:,710.8
Df Model:,10,,
Covariance Type:,nonrobust,,


In [159]:
ols_model10.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.1535,0.152,-1.008,0.314,-0.453,0.146
treat2016,0.0258,0.067,0.388,0.698,-0.105,0.156
yr_2016,-0.0054,0.045,-0.121,0.904,-0.094,0.083
treatment_class,-0.0305,0.046,-0.660,0.509,-0.121,0.060
female_prof,0.0201,0.034,0.591,0.555,-0.047,0.087
instate,-0.0037,0.040,-0.094,0.925,-0.082,0.074
freshman,0.0243,0.049,0.491,0.623,-0.073,0.121
american,-0.0431,0.057,-0.758,0.449,-0.155,0.069
ACumGPA,0.0967,0.056,1.742,0.082,-0.012,0.206
gradePrinciples,0.0137,0.028,0.495,0.621,-0.041,0.068


In [161]:
print(ols_model10.summary())

                            OLS Regression Results                            
Dep. Variable:         Major_Business   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.416
Date:                Sat, 15 Jul 2023   Prob (F-statistic):              0.169
Time:                        22:09:20   Log-Likelihood:                -319.96
No. Observations:                 627   AIC:                             661.9
Df Residuals:                     616   BIC:                             710.8
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.1535      0.152     

In [163]:
# Encontramos standar  Error  robusto de Huber-White

ols_model10_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model10_rb.summary())

                            OLS Regression Results                            
Dep. Variable:         Major_Business   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     78.36
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           9.36e-09
Time:                        22:09:39   Log-Likelihood:                -319.96
No. Observations:                 627   AIC:                             661.9
Df Residuals:                     616   BIC:                             710.8
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.1535      0.214     

In [164]:
#Cuarta regresión, contiene variable de control

y = df['Major_Marketing']

# agregamos la constante
## "Major_Marketing": Major marketing
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X

,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [165]:

ols_model11 = sm.OLS(y, X).fit()

In [166]:
ols_model11.summary().tables[0]

Dep. Variable:,Major_Marketing,R-squared:,0.037
Model:,OLS,Adj. R-squared:,0.022
Method:,Least Squares,F-statistic:,2.389
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.00877
Time:,22:10:41,Log-Likelihood:,-199.30
No. Observations:,627,AIC:,420.6
Df Residuals:,616,BIC:,469.5
Df Model:,10,,
Covariance Type:,nonrobust,,


In [167]:
ols_model11.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.2548,0.126,-2.028,0.043,-0.501,-0.008
treat2016,-0.0176,0.055,-0.320,0.749,-0.125,0.090
yr_2016,0.0286,0.037,0.773,0.440,-0.044,0.101
treatment_class,0.0370,0.038,0.973,0.331,-0.038,0.112
female_prof,0.0519,0.028,1.847,0.065,-0.003,0.107
instate,-0.0600,0.033,-1.828,0.068,-0.124,0.004
freshman,0.0498,0.041,1.222,0.222,-0.030,0.130
american,0.1126,0.047,2.402,0.017,0.021,0.205
ACumGPA,0.0467,0.046,1.020,0.308,-0.043,0.137
gradePrinciples,0.0154,0.023,0.676,0.499,-0.029,0.060


In [169]:
print(ols_model11.summary())

                            OLS Regression Results                            
Dep. Variable:        Major_Marketing   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     2.389
Date:                Sat, 15 Jul 2023   Prob (F-statistic):            0.00877
Time:                        22:11:20   Log-Likelihood:                -199.30
No. Observations:                 627   AIC:                             420.6
Df Residuals:                     616   BIC:                             469.5
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.2548      0.126     

In [171]:
# Encontramos standar Error robusto de Huber-White

ols_model11_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model11_rb.summary())

                            OLS Regression Results                            
Dep. Variable:        Major_Marketing   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     305.1
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           5.81e-12
Time:                        22:11:39   Log-Likelihood:                -199.30
No. Observations:                 627   AIC:                             420.6
Df Residuals:                     616   BIC:                             469.5
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.2548      0.093     

In [172]:

# Variables  explicativas a mostrarse en la tabla

explicativas = ['treat2016','yr_2016','treatment_class','const']

# etiquetas para las variables 

etiquetas = ['Treatment class × 2016','Year 2016','Treatment class (in 2015)','Constant']


labels = dict(zip(explicativas,etiquetas))
labels 

{'treat2016': 'Treatment class × 2016',
 'yr_2016': 'Year 2016',
 'treatment_class': 'Treatment class (in 2015)',
 'const': 'Constant'}

In [211]:
#Pasamos la  tabla 5 a Latex

pystout(models = [ols_model8,ols_model9,ols_model10,ols_model11], file=r"..\..\Trabajo_final\Grupo9\tabla5.tex", digits=3,
        endog_names=['Major STEM','Major finance','Major business','Major marketing'],
        exogvars =explicativas ,  
        varlabels = labels,
        modstat={'nobs':'Observations'}, 
        addrows={'Controls':['Yes','Yes','Yes','Yes']},  
        addnotes=['Notes: LPM regressions. We report wild bootstrap cluster p-values in parentheses and wild bootstrap cluster 95 per-\
        cent confidence intervals in square brackets, generated using boottest command in Stata 14 (Roodman et al. 2019)\
        for standard errors clustered at the class level (12 clusters). Dependent variables: dummy equal to one if the student\
        majored in (i) STEM subjects, (ii) finance, (iii) business, or (iv) marketing.', 
                 '* Significantly different from zero at 90 percent confidence.',
                 '** Significantly different from zero at 95 percent confidence.',
                 '*** Significantly different from zero at 99 percent confidence.'],
        title='Treatment Effects on Other High-Earning Majors',
        stars={.1:'*',.05:'**',.01:'***'}
       )

In [174]:
# Tabla 6 
##Primera regresión, contiene variable de control

y = df['Major_SocSc']

# agregamos la constante
## "Major_SocSc": Major social sciences
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X


,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [175]:
# Utilizamos la Función Homocedastica 
ols_model12 = sm.OLS(y, X).fit()

In [176]:
ols_model12.summary().tables[0]

Dep. Variable:,Major_SocSc,R-squared:,0.078
Model:,OLS,Adj. R-squared:,0.063
Method:,Least Squares,F-statistic:,5.209
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,2.32e-07
Time:,22:13:02,Log-Likelihood:,-131.82
No. Observations:,627,AIC:,285.6
Df Residuals:,616,BIC:,334.5
Df Model:,10,,
Covariance Type:,nonrobust,,


In [177]:
ols_model12.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.6238,0.113,5.530,0.000,0.402,0.845
treat2016,-0.0062,0.049,-0.126,0.899,-0.103,0.091
yr_2016,-0.0230,0.033,-0.691,0.490,-0.088,0.042
treatment_class,-0.0369,0.034,-1.080,0.281,-0.104,0.030
female_prof,-0.0359,0.025,-1.422,0.155,-0.086,0.014
instate,-0.0278,0.029,-0.946,0.345,-0.086,0.030
freshman,-0.0780,0.037,-2.132,0.033,-0.150,-0.006
american,0.0896,0.042,2.127,0.034,0.007,0.172
ACumGPA,-0.1105,0.041,-2.687,0.007,-0.191,-0.030
gradePrinciples,-0.0298,0.020,-1.458,0.145,-0.070,0.010


In [178]:

ols_model12.summary().tables[2]

Omnibus:,277.055,Durbin-Watson:,2.028
Prob(Omnibus):,0.000,Jarque-Bera (JB):,890.075
Skew:,2.224,Prob(JB):,5.28e-194
Kurtosis:,6.780,Cond. No.,49.5


In [179]:
print(ols_model12.summary())

                            OLS Regression Results                            
Dep. Variable:            Major_SocSc   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     5.209
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           2.32e-07
Time:                        22:13:31   Log-Likelihood:                -131.82
No. Observations:                 627   AIC:                             285.6
Df Residuals:                     616   BIC:                             334.5
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.6238      0.113     

In [181]:
# Encontramos standar Error  robusto de Huber-White

ols_model12_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model12_rb.summary())

                            OLS Regression Results                            
Dep. Variable:            Major_SocSc   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     1483.
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           9.97e-16
Time:                        22:13:48   Log-Likelihood:                -131.82
No. Observations:                 627   AIC:                             285.6
Df Residuals:                     616   BIC:                             334.5
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.6238      0.156     

In [182]:
#Segunda regresión, contiene variable de control

y = df['Major_Arts']

# agregamos la constante
## "Major_Arts": Major arts
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X


,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [183]:

ols_model13 = sm.OLS(y, X).fit()

In [184]:
ols_model13.summary().tables[0]

Dep. Variable:,Major_Arts,R-squared:,0.022
Model:,OLS,Adj. R-squared:,0.006
Method:,Least Squares,F-statistic:,1.382
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.185
Time:,22:14:34,Log-Likelihood:,48.480
No. Observations:,627,AIC:,-74.96
Df Residuals:,616,BIC:,-26.11
Df Model:,10,,
Covariance Type:,nonrobust,,


In [185]:
ols_model13.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.0503,0.085,0.595,0.552,-0.116,0.216
treat2016,-0.0312,0.037,-0.844,0.399,-0.104,0.041
yr_2016,0.0180,0.025,0.721,0.471,-0.031,0.067
treatment_class,0.0354,0.026,1.381,0.168,-0.015,0.086
female_prof,-0.0189,0.019,-0.999,0.318,-0.056,0.018
instate,0.0176,0.022,0.795,0.427,-0.026,0.061
freshman,-0.0082,0.027,-0.301,0.764,-0.062,0.046
american,-0.0048,0.032,-0.151,0.880,-0.067,0.057
ACumGPA,0.0368,0.031,1.194,0.233,-0.024,0.097
gradePrinciples,-0.0434,0.015,-2.834,0.005,-0.074,-0.013


In [187]:
print(ols_model13.summary())

                            OLS Regression Results                            
Dep. Variable:             Major_Arts   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.382
Date:                Sat, 15 Jul 2023   Prob (F-statistic):              0.185
Time:                        22:15:10   Log-Likelihood:                 48.480
No. Observations:                 627   AIC:                            -74.96
Df Residuals:                     616   BIC:                            -26.11
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0503      0.085     

In [189]:
# Encontramos standar Error  robusto de Huber-White

ols_model13_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model13_rb.summary())

                            OLS Regression Results                            
Dep. Variable:             Major_Arts   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     5.253
Date:                Sat, 15 Jul 2023   Prob (F-statistic):            0.00566
Time:                        22:18:06   Log-Likelihood:                 48.480
No. Observations:                 627   AIC:                            -74.96
Df Residuals:                     616   BIC:                            -26.11
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0503      0.070     

In [190]:
#Tercera regresión, contiene variable de control

y = df['Major_Comm']

# agregamos la constante
## "Major_Comm": Major communication
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X


,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [191]:
ols_model14 = sm.OLS(y, X).fit()

In [192]:
ols_model14.summary().tables[0]

Dep. Variable:,Major_Comm,R-squared:,0.021
Model:,OLS,Adj. R-squared:,0.005
Method:,Least Squares,F-statistic:,1.336
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.207
Time:,22:18:47,Log-Likelihood:,31.395
No. Observations:,627,AIC:,-40.79
Df Residuals:,616,BIC:,8.060
Df Model:,10,,
Covariance Type:,nonrobust,,


In [193]:

ols_model14.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.1463,0.087,1.683,0.093,-0.024,0.317
treat2016,-0.0119,0.038,-0.314,0.754,-0.087,0.063
yr_2016,-0.0077,0.026,-0.302,0.763,-0.058,0.043
treatment_class,-0.0020,0.026,-0.074,0.941,-0.054,0.050
female_prof,-0.0268,0.019,-1.376,0.169,-0.065,0.011
instate,-0.0067,0.023,-0.295,0.768,-0.051,0.038
freshman,0.0417,0.028,1.480,0.140,-0.014,0.097
american,0.0553,0.032,1.704,0.089,-0.008,0.119
ACumGPA,-0.0427,0.032,-1.346,0.179,-0.105,0.020
gradePrinciples,-0.0003,0.016,-0.021,0.983,-0.031,0.031


In [194]:
ols_model14.summary().tables[2]

Omnibus:,489.288,Durbin-Watson:,1.978
Prob(Omnibus):,0.000,Jarque-Bera (JB):,5127.813
Skew:,3.681,Prob(JB):,0.00
Kurtosis:,14.920,Cond. No.,49.5


In [195]:
print(ols_model14.summary())

                            OLS Regression Results                            
Dep. Variable:             Major_Comm   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.336
Date:                Sat, 15 Jul 2023   Prob (F-statistic):              0.207
Time:                        22:19:10   Log-Likelihood:                 31.395
No. Observations:                 627   AIC:                            -40.79
Df Residuals:                     616   BIC:                             8.060
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1463      0.087     

In [197]:
# Encontramos standar Error robusto de Huber-White

ols_model14_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model14_rb.summary())

                            OLS Regression Results                            
Dep. Variable:             Major_Comm   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     20.19
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           1.15e-05
Time:                        22:19:28   Log-Likelihood:                 31.395
No. Observations:                 627   AIC:                            -40.79
Df Residuals:                     616   BIC:                             8.060
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1463      0.073     

In [198]:
#Cuarta regresión, contiene variable de control

y = df['Major_Hum']

# agregamos la constante
## "Major_Hum": Major humanities
## "treat2016": Treatment class × 2016
## "yr_2016": Year 2016
X = sm.add_constant(df.loc[:,["treat2016", "yr_2016", "treatment_class", "female_prof", "instate", "freshman", "american", "ACumGPA",
"gradePrinciples", "small_class"]])
X


,const,treat2016,yr_2016,treatment_class,female_prof,instate,freshman,american,ACumGPA,gradePrinciples,small_class
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.267,4.00,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.358,3.00,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.775,1.67,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3.654,3.00,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.000,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
622,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.653,2.33,0.0
623,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.200,0.33,0.0
624,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.677,3.67,1.0
625,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.500,3.00,0.0


In [199]:
# Utilizamos Función Homocedastica 
ols_model15 = sm.OLS(y, X).fit()

In [200]:
ols_model15.summary().tables[0]

Dep. Variable:,Major_Hum,R-squared:,0.038
Model:,OLS,Adj. R-squared:,0.023
Method:,Least Squares,F-statistic:,2.445
Date:,"Sat, 15 Jul 2023",Prob (F-statistic):,0.00726
Time:,22:20:03,Log-Likelihood:,-69.809
No. Observations:,627,AIC:,161.6
Df Residuals:,616,BIC:,210.5
Df Model:,10,,
Covariance Type:,nonrobust,,


In [201]:
ols_model15.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.1404,0.102,1.374,0.170,-0.060,0.341
treat2016,-0.1413,0.045,-3.165,0.002,-0.229,-0.054
yr_2016,0.0671,0.030,2.227,0.026,0.008,0.126
treatment_class,0.0828,0.031,2.677,0.008,0.022,0.144
female_prof,-0.0449,0.023,-1.961,0.050,-0.090,6.14e-05
instate,0.0286,0.027,1.071,0.285,-0.024,0.081
freshman,-0.0073,0.033,-0.221,0.825,-0.072,0.058
american,0.0201,0.038,0.527,0.598,-0.055,0.095
ACumGPA,0.0069,0.037,0.186,0.852,-0.066,0.080
gradePrinciples,-0.0412,0.019,-2.225,0.026,-0.078,-0.005


In [203]:
print(ols_model15.summary())

                            OLS Regression Results                            
Dep. Variable:              Major_Hum   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     2.445
Date:                Sat, 15 Jul 2023   Prob (F-statistic):            0.00726
Time:                        22:20:38   Log-Likelihood:                -69.809
No. Observations:                 627   AIC:                             161.6
Df Residuals:                     616   BIC:                             210.5
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1404      0.102     

In [204]:
# Encontraamos standar error robusto de Huber-White

ols_model15_rb = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['class_fe2']})
print(ols_model15_rb.summary())

                            OLS Regression Results                            
Dep. Variable:              Major_Hum   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     18.66
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           1.71e-05
Time:                        22:20:52   Log-Likelihood:                -69.809
No. Observations:                 627   AIC:                             161.6
Df Residuals:                     616   BIC:                             210.5
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1404      0.115     

In [206]:
#Variables explicativas a mostrarse en la tabla

explicativas = ['treat2016','yr_2016','treatment_class','const']

# etiquetas para las variables 

etiquetas = ['Treatment class × 2016','Year 2016','Treatment class (in 2015)','Constant']


labels = dict(zip(explicativas,etiquetas))
labels 

{'treat2016': 'Treatment class × 2016',
 'yr_2016': 'Year 2016',
 'treatment_class': 'Treatment class (in 2015)',
 'const': 'Constant'}

In [210]:
# pasamos la  tabla 6 a Latex

pystout(models = [ols_model12,ols_model13,ols_model14,ols_model15], file=r"..\..\Trabajo_final\Grupo9\tabla6.tex", digits=3,
        endog_names=['Major social sciences','Major arts','Major communication','Major humanities'],
        exogvars =explicativas ,  
        varlabels = labels,
        modstat={'nobs':'Observations'}, 
        addrows={'Controls':['Yes','Yes','Yes','Yes']},  
        addnotes=['Notes: LPM regressions. We report wild bootstrap cluster p-values in parentheses and wild bootstrap cluster 95 per-\
        cent confidence intervals in square brackets, generated using boottest command in Stata 14 (Roodman et al. 2019)\
        for standard errors clustered at the class level (12 clusters). Dependent variables: dummy equal to one if the stu-\
        dent majored in (i) social science (other than economics), (ii) arts, (iii) communication studies, or (iv) humanities.',
                 '* Significantly different from zero at 90 percent confidence.',
                 '** Significantly different from zero at 95 percent confidence.',
                 '*** Significantly different from zero at 99 percent confidence.'],
        title='Treatment Effects on Other High-Earning Majors',
        stars={.1:'*',.05:'**',.01:'***'}
       )